In [1]:
import gurobipy as gp
import numpy as np
from gurobipy import GRB

In [2]:
zu = np.inf
zl = 0

In [3]:
A = np.array([
    [3, 1, 2, 4, 1],
    [2, 3, 1, 4, 2]
])
b = np.array([6, 5])

In [5]:
np.sum(A, axis=-1)

array([11, 12])

In [ ]:
A.T[0]

In [ ]:
lambd = np.zeros(A.shape[0])
lambd

In [ ]:
Ct = [sum([(1 - lambd[r]) * A[r][c] for r in range(b.shape[0])]) 
      for c in range(A.shape[-1])]
Ct

In [ ]:
x = [0 if ct >= 0 else 1 for ct in Ct]
x

In [ ]:
C_t = (1 - lambd) @ A
x = np.where(C_t < 0, 1, 0)
C_t

In [ ]:
L = C_t @ x + lambd @ b
L

In [ ]:
zl = max(zl, L)
zl

In [ ]:
g = b - A @ x
g

In [ ]:
A = np.array([
    [1, 2, 4],
    [3, 1, 4],
    [2, 2, 2]
])
b = np.array([2, 5, 1])

In [ ]:
def subgrad_opt(
        A, b, z_ub, z_lb, lambd, f=2, k=15, eps=0.005, omega=500):
    # TODO: z_ub cannot be inf here
    unchanged = 0
    t = 0
    lambd_best = lambd
    x_best = None
    while (z_ub > z_lb):
        c = (1 - lambd) @ A
        # print("c: ", c)
        x = np.where(c < 0, 1, 0)
        # print(x)
        L = c @ x + lambd @ b
        # print("L: ", L)
        g = b - A @ x
        if L > z_lb:
            z_lb = L
            lambd_best = lambd
            x_best = x
            unchanged = 0
            # print("L new: ", L)
        else:
            unchanged += 1
        if unchanged == k:
            unchanged = 0
            f /= 2
        sigma = f * (z_ub - z_lb) / np.linalg.norm(g) ** 2
        lambd = np.maximum(np.zeros_like(lambd), lambd + sigma * g)
        # lambd = np.minimum(1, lambd)
        # print("lambd: ", lambd)
        t += 1
        if f < eps or t > omega:
            print(f, t, g)
            break
    
    return lambd_best, z_lb, x_best, f

In [ ]:
subgrad_opt(A, b, 100, 0, np.zeros(A.shape[0]))

# RCH

In [ ]:
def rch(A, b):
    lambd = np.zeros(A.shape[0])
    x = np.zeros(A.shape[-1])
    s = (1 - lambd) @ A
    print("s", s)
    for j in range(A.shape[-1]):
        if s[j] == 0:
            x[j] = 1
        else:
            x[j] = 0

    print("x", x)
    
    for i in range(A.shape[0]):
        N_i = np.where(A[i] > 0)[0]
        print("i N_i", i, N_i)
        if sum([A[i][j] for j in N_i]) < b[i]:
            s_k = min(s[N_i])
            lambd[i] += s_k
            for j in N_i:
                s[j] -= s_k
                if s[j] == 0:
                    x[j] = 1

    print(s, x)

    return lambd, make_prime(x, A, b)

# RCH not working because it assumes that only one column needs to cover

def make_prime(x, A, b):
    x1 = np.where(x == 1)[0]
    for j in x1:
        new_x1 = x1[~np.isin(x1, [j])]
        if A @ new_x1 >= b:
            x1 = new_x1
            print("changed")
    return x1


In [ ]:
rch(A, b)

# Solve dual

In [ ]:
c = np.sum(A, axis=0)
c

In [ ]:
md = gp.Model("dual")
y = md.addMVar(A.shape[0], lb=0, name="y")
md.setObjective(b @ y, GRB.MAXIMIZE)
constr = md.addConstr(A.T @ y <= c)
md.update()

In [ ]:
md

In [ ]:
md.remove(constr[0])
md.update()
md

In [ ]:
md.optimize()
res = []
for v in md.getVars():
    res.append(v.x)
md.getObjective().getValue(), res

In [ ]:
f=2 
k=5 
eps=0.005 
omega=500
ub = 90
x0 = []
x1 = []
lambd = np.zeros(A.shape[0])
lb = -np.inf

unchanged = 0
t = 0
lambd_best = lambd
x_best = None
while (ub > lb):
    c = (1 - lambd) @ A
    x = np.where(c < 0, 1, 0)
    x[x0] = 0
    x[x1] = 1
    L = c @ x + lambd @ b
    g = b - A @ x
    if L > lb:
        lb = L
        lambd_best = lambd
        x_best = x
        unchanged = 0
    else:
        unchanged += 1
    if unchanged == k:
        unchanged = 0
        f /= 2
    sigma = f * (ub - lb) / np.linalg.norm(g) ** 2
    lambd = np.maximum(
        np.zeros_like(lambd), lambd + sigma * g)
    t += 1
    if f < eps or t > omega:
        break

In [ ]:
lambd_best

# Dual ascent

In [50]:
c = np.sum(A, axis=0)
# s = (1 - lambd) @ A
s = c
lambd = np.zeros((A.shape[0]))
N = []
for i in range(A.shape[0]):
    idx = np.where(A[i] > 0)[0]
    N.append((i, idx))
N


[(0, array([0, 1, 2, 3, 4], dtype=int64)),
 (1, array([0, 1, 2, 3, 4], dtype=int64))]

In [51]:
list.sort(N, key=lambda x: x[0])
print(N)

[(0, array([0, 1, 2, 3, 4], dtype=int64)), (1, array([0, 1, 2, 3, 4], dtype=int64))]


In [52]:
for i, _ in N:
    lambd[i] = min([s[j] for j in N[i][1]])
    for j in range(A.shape[-1]):
        if j in N[i][1]:
            s[j] -= lambd[i]
lambd

array([3., 0.])

# Dual ascent 2

In [49]:
c = np.sum(A, axis=0)
lambd = np.zeros((A.shape[0]))
s = (1 - lambd) @ A
N = []
for i in range(A.shape[0]):
    idx = np.where(A[i] > 0)[0]
    N.append((i, idx))

list.sort(N, key=lambda x: x[0], reverse=True)

for i, _ in N:
    lambd[i] = max(0, lambd[i] + min(0, min([s[j] for j in N[i][1]])))

list.sort(N, key=lambda x: x[0])

for i, _ in N:
    lambd[i] = lambd[i] + min([s[j] for j in N[i][1]])

lambd

array([3., 3.])